In [23]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
import qutip as qt
import scipy as sp
import sympy as sym


%matplotlib ipympl
plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=300, formatter={'float': '{:.4f}'.format})
GHz = 1e9

In [55]:
def get_unique_energy_indices_generalized(E_combined, E_elements):
    indices_combinations = list(product(*[range(len(e)) for e in E_elements]))
    
    total_energies = [sum(E_elements[i][indices[i]] for i in range(len(E_elements))) for indices in indices_combinations]
    
    # Dictionary to store already found index combinations for each energy level
    used_combinations = {E: [] for E in set(E_combined)}
    
    matched_indices = []
    for E in E_combined:
        differences = np.abs(np.array(total_energies) - E)
        sorted_diff_indices = np.argsort(differences)
        
        # Find the closest match that has not been used yet
        for idx in sorted_diff_indices:
            if indices_combinations[idx] not in used_combinations[E]:
                matched_indices.append(indices_combinations[idx])
                used_combinations[E].append(indices_combinations[idx])
                break
        else:
            # If all combinations have been used or no match found within tolerance
            matched_indices.append(tuple([-123] * len(E_elements)))

    return np.array(matched_indices)


def correctly_reorganize_matched_indices(E_combined, matched_indices):
    # Identify degenerate energy levels and their indices
    unique, counts = np.unique(E_combined, return_counts=True)
    degenerate_levels = unique[counts > 1]

    for level in degenerate_levels:
        degenerate_indices = np.where(E_combined == level)[0]
        # Extract the matched indices for these degenerate levels
        degenerate_matched_indices = matched_indices[degenerate_indices]
        
        # Define a custom sort function that prioritizes the first non-zero index
        def sort_key(x):
            for i, xi in enumerate(x):
                if xi > 0:
                    return (i, xi)
            return (len(x), 0)  # Handle the case where all elements are zero

        # Sort based on the custom key
        sorted_indices = sorted(range(len(degenerate_matched_indices)), key=lambda i: sort_key(degenerate_matched_indices[i]))
        
        # Reassign sorted matched indices back to the original array
        matched_indices[degenerate_indices] = degenerate_matched_indices[sorted_indices]

    return matched_indices


E_combined = np.array([0, 1, 1, 2, 2, 2])
E_elements = [np.array([0, 1, 2]), np.array([0, 1, 2]), np.array([0, 2, 4])]

# First, obtain matched indices using the existing approach
matched_indices = get_unique_energy_indices_generalized(E_combined, E_elements)

# Then, reorganize matched indices for degenerate energy levels
reorganized_matched_indices = correctly_reorganize_matched_indices(E_combined, matched_indices)
reorganized_matched_indices

array([[0, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [2, 0, 0],
       [0, 2, 0],
       [0, 0, 1]])

In [26]:
sq_ext.diag(sq_ext.hamiltonian_qubit(), 6, out='GHz', remove_ground=True)[0]

array([0.0000, 1.9454, 6.1611, 7.2085, 9.1539, 10.7929])

In [27]:
sq_ext.diag(sq_ext.hamiltonian_qubit_C_qubit(nmax_r=5, nmax_f=10, Cc=0), 8, out='GHz', remove_ground=True)[0]

array([0.0000, 1.9452, 1.9452, 3.8905, 6.1597, 6.1597, 7.2085, 7.2085])

In [10]:
E_f = sq_ext.diag(sq_ext.sq_fluxonium().hamiltonian(),4, out='GHz')[0]
E_f -= E_f[0]
E_r = sq_ext.diag(sq_ext.sq_resonator().hamiltonian(),4, out='GHz')[0]
E_r -= E_r[0]

In [11]:
E_matrix = E_f[:, np.newaxis] + E_r
E_matrix

array([[0.0000, 7.2084, 14.4168, 21.6252],
       [1.9455, 9.1539, 16.3623, 23.5707],
       [6.1613, 13.3697, 20.5781, 27.7865],
       [10.7934, 18.0018, 25.2101, 32.4185]])

In [12]:
E_f[:, np.newaxis]

array([[0.0000],
       [1.9455],
       [6.1613],
       [10.7934]])

In [1182]:
fluxonium = sq_ext.sq_fluxonium()
H_0_f = fluxonium.hamiltonian() 
H_0 = qt.tensor( H_0_f, qt.qeye(H_0_f.shape[0]) ) + qt.tensor(qt.qeye(H_0_f.shape[0]), H_0_f ) #+ 0.00001*qt.tensor(fluxonium.flux_op(0), fluxonium.flux_op(0))
H = qt.tensor( H_0_f, qt.qeye(H_0_f.shape[0]) ) + qt.tensor(qt.qeye(H_0_f.shape[0]), H_0_f  )+ 1e5*qt.tensor(fluxonium.flux_op(0), fluxonium.flux_op(0))

In [1266]:
E, ψ_0_f = np.linalg.eigh(H_0_f.__array__() )
# E, ψ_0_f = sp.sparse.linalg.eigsh(H_0_f.data, 2, which='SA')
E, ψ_0_f = sq_ext.eigs_sorted(E, ψ_0_f)
ψ_0 = np.array([np.kron(ψ_0_f[:,0],ψ_0_f[:,0]), np.kron(ψ_0_f[:,1],ψ_0_f[:,0]), np.kron(ψ_0_f[:,0],ψ_0_f[:,1]), np.kron(ψ_0_f[:,1],ψ_0_f[:,1]) ]).T

In [1219]:
E, ψ_0 = sp.sparse.linalg.eigsh(H_0.data, 2, which='SA')
E, ψ_0 = sq_ext.eigs_sorted(E, ψ_0)
ψ_0 = np.linalg.qr(ψ_0)[0]

In [1256]:
E, ψ_0 = np.linalg.eigh(H_0.__array__())
ψ_0 =  ψ_0[:,:4]

In [1262]:
np.round(ψ_0.T.conj() @ ψ_0,4)

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.-0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.-0.j, 0.-0.j, 1.+0.j, 0.+0.j],
       [0.-0.j, 0.-0.j, 0.-0.j, 1.+0.j]])

In [1267]:
H_eff = ψ_0.T.conj() @ H.__array__() @ ψ_0 / 2 / np.pi / GHz
sq_ext.decomposition_in_pauli_4x4(H_eff, 4);

 (15.3596+0j)	*	 I ⨂I 
 (-0.9727+0j)	*	 I ⨂σz 
 (0.0235+0j)	*	 σx ⨂σx 
 (-0.9727+0j)	*	 σz ⨂I 


In [1161]:
H_eff = sq_ext.H_eff_p1(H_0, H, out= 'GHz', n_eig=4, solver='numpy')
# H_eff = sq_ext.H_eff_p1(H_0, H, out= 'GHz', n_eig=4)
sq_ext.decomposition_in_pauli_4x4(H_eff, 4);

 (15.3596+0j)	*	 I ⨂I 
 (-0.961+0j)	*	 I ⨂σz 
 (0.0125+0j)	*	 σx ⨂σx 
 (-0.011+0j)	*	 σy ⨂σy 
 (-0.9845+0j)	*	 σz ⨂I 


In [1106]:
ψ_0[:,2] /= np.exp(1j*0.1)
H_eff = ψ_0.conj().T @ H.__array__() @ ψ_0 / 2 / np.pi / GHz
sq_ext.decomposition_in_pauli_4x4(H_eff, 4);

ValueError: shapes (4,4) and (625,625) not aligned: 4 (dim 1) != 625 (dim 0)